

# Коллоборативная фильтрация. Факторизация матриц

Все началось с конкурса netflix: https://www.netflixprize.com/leaderboard.html

Задача - предсказать какой рейтинг пользователь поставит фильму

Приз - 1 млн долларов

Метрика - RMSE

Задача - улучшить RMSE полученную текущей моделью Netflix на 10%

<img src='netflix_progress.jpg'>

<img src='mf.png'>

Для любого пользователя можно предсказать оценки, которые он еще не ставил. Обучать модель нужно так, чтобы она давала наименьшие ошибки для уже известных моделей.

Предсказание - скалярное произведение вектора для пользователя и вектора для объекта.

При таком подходе необходимо обучить намного меньше параметров модели.

размерность матриц:

- R - n_users x n_items
- P - n_users x n_factors
- Q - n_factors x n_items

n_factors - гиперпараметр, мы сами выбираем

Пример:
- n_users = 1000
- n_items = 2000
- n_factors = 10

- R - 2 000 000 параметров
- P, Q - 30 000 параметров

SVD напрямую применить нельзя, так как очень много отсутствующих значений (точнее можно, как-то заполнив их, но получается не очень хорошее качество).

Поэтому формулируется задача оптимизации по известным оценкам $K$:

### SVD no bias

$r̂_{ui}= q_i^T p_u$

$ min_{p, q} \sum_{u,i \in K} (r_{ui} - q_i^T p_u)^2 + \lambda (||q_i||^2 + ||p_u||^2)$

Как решают эту проблему оптимизации:

1) SGD - берем производные  по $p_u$ и $q_i$

2) ALS - Alternating least squares:
- зафиксируем $p_u$ - задача имеет аналитическое решение для $q_i$ ( метод наименьших квадратов с регурялизацией)
- зафиксируем $q_i$ - задача имеет аналитическое решение для $p_u$

3) Markov Chain Monte Carlo (MCMC) Inference

### SVD biased

$r̂_{ui} = μ + b_u + b_i + q_i^T p_u$

μ - глобальный средний рейтинг

b_u - средний рейтинг для пользователя

b_i - средний рейтинг для item

$ min_{p, q} \sum_{u,i \in K} (r_{ui} - (μ + b_u + b_i + q_i^T p_u))^2 + \lambda (b_i^2+b_u^2 + ||q_i||^2 + ||p_u||^2)$

## Пример

данные с оценками фильмов:

https://grouplens.org/datasets/movielens/100k/

https://www.kaggle.com/prajitdatta/movielens-100k-dataset

библиотеки:
- pandas
- surprise
- numpy
- fastFM
- git+git://github.com/scikit-learn/scikit-learn.git


Установить можно так:
```bash
pip install -r requirements.txt
```

In [2]:
%load_ext autoreload
%autoreload 2

In [8]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate, GridSearchCV
from surprise import Reader, Dataset, SVD, evaluate
import numpy as np

## Чтение данных в surprise

In [9]:
# Load the movielens-100k dataset (download it if needed).
data = Dataset.load_builtin('ml-100k')

# или можно загрузить данные из файла / пандас data frame
data = Dataset.load_from_file('ml-100k/ua.base', reader=Reader(sep='\t'))

In [5]:
# Load the movielens-100k dataset (download it if needed).
data = Dataset.load_builtin('ml-100k')

# # Use the famous SVD algorithm.
svd_bias = SVD()
svd_no_bias = SVD(biased=False)

## Сравним biased и not biased версию SVD

In [7]:
# Run 5-fold cross-validation and print results.
cross_validate(svd_bias, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9317  0.9337  0.9368  0.9436  0.9404  0.9372  0.0043  
MAE (testset)     0.7349  0.7328  0.7367  0.7437  0.7407  0.7378  0.0039  
Fit time          7.71    8.13    6.80    8.21    7.30    7.63    0.53    
Test time         0.43    0.20    0.19    0.31    0.19    0.26    0.09    


{'test_rmse': array([0.93170947, 0.93368729, 0.93680563, 0.94356846, 0.94042431]),
 'test_mae': array([0.73494358, 0.7327895 , 0.73672189, 0.74366837, 0.74072947]),
 'fit_time': (7.709680795669556,
  8.132670879364014,
  6.79968786239624,
  8.20711612701416,
  7.2996437549591064),
 'test_time': (0.42635011672973633,
  0.19845032691955566,
  0.18872594833374023,
  0.3060462474822998,
  0.18769001960754395)}

In [8]:
# Run 5-fold cross-validation and print results.
cross_validate(svd_no_bias, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9526  0.9532  0.9516  0.9442  0.9444  0.9492  0.0040  
MAE (testset)     0.7513  0.7497  0.7512  0.7426  0.7430  0.7476  0.0039  
Fit time          8.14    6.72    6.79    6.93    7.14    7.14    0.52    
Test time         0.16    0.25    0.17    0.31    0.37    0.25    0.08    


{'test_rmse': array([0.95260899, 0.95322096, 0.95157809, 0.94424367, 0.94440674]),
 'test_mae': array([0.75125894, 0.74972678, 0.75121073, 0.74261804, 0.74301369]),
 'fit_time': (8.143501996994019,
  6.719550132751465,
  6.7867021560668945,
  6.926789999008179,
  7.143996953964233),
 'test_time': (0.16225695610046387,
  0.2450881004333496,
  0.16622209548950195,
  0.3088991641998291,
  0.36506009101867676)}

# Найдем оптимальные параметры и проверим ошибку на test set

In [12]:
param_grid = {
    'lr_all': [0.005, 0.05],
    'reg_all': [0.02, 0.002],
    'n_factors': [5, 10, 100, 500],
    'n_epochs': [10, 100]
}

gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

gs.fit(data)

# best RMSE score
print(gs.best_score['rmse'])

In [ ]:
print(gs.best_score)

In [ ]:
gs.best_params

In [14]:
svd = SVD(**gs.best_params['rmse'])

In [15]:
from surprise.accuracy import mae, rmse

In [10]:
data_train = Dataset.load_from_file('ml-100k/ua.base', reader=Reader(sep='\t'))
data_test = Dataset.load_from_file('ml-100k/ua.test', reader=Reader(sep='\t'))

train_set = data_train.build_full_trainset()
test_set = data_test.build_full_trainset().build_testset()

svd = SVD()

In [17]:
svd.train(train_set)

/Users/nazhmeddinbabakhanov/anaconda3/lib/python3.6/site-packages/surprise/prediction_algorithms/algo_base.py:51: UserWarning: train() is deprecated. Use fit() instead
  warnings.warn('train() is deprecated. Use fit() instead', UserWarning)


In [18]:
pred = svd.test(test_set)
mae(pred), rmse(pred)

MAE:  0.7565
RMSE: 0.9533


(0.7565499447581555, 0.953310571577554)

## то же самое, но через метод predict 

In [19]:
from sklearn.metrics import mean_squared_error, mean_absolute_error


targets, preds = zip(*[(t, svd.predict(u_id, i_id).est) for u_id, i_id, t, _ in data_test.raw_ratings])
print('sklearn MAE: {:.4f}'.format(mean_absolute_error(targets, preds)))
print('sklearn RMSE: {:.4f}'.format(np.sqrt(mean_squared_error(targets, preds))))

sklearn MAE: 0.7565
sklearn RMSE: 0.9533


In [20]:
data_train.raw_ratings[:2]

[('1', '1', 5.0, None), ('1', '2', 3.0, None)]

In [21]:
svd.default_prediction()

3.5238268742409184

In [22]:
print(svd.predict('1', '1'))
print(svd.predict(1, 1))
print('default prediction: {:.2f}'.format(svd.default_prediction()))

user: 1          item: 1          r_ui = None   est = 3.75   {'was_impossible': False}
user: 1          item: 1          r_ui = None   est = 3.52   {'was_impossible': False}
default prediction: 3.52


# Обобщение идеи. Факторизационные машины

Метод SVD рассмотренный выше мы применяли только к матрице user - item.

Что если мы хотим учесть какие-то другие факторы для предсказания рейтинга:
- implicit feedback (учесть страницы фильмов на которые заходил пользователь, но не поставил рейтинг)
- учесть как предпочтения менялись со временем
- учесть дополнительную информацию о продуктах (жанр фильма)
- учесть дополнительную инфо о пользователя (возрастная группа, пол)

Очень много kaggle соревнований на рекомендации были выиграны с помощью факторизационных машин


<img src='fm.png'>


источник: https://www.csie.ntu.edu.tw/~b97053/paper/Factorization%20Machines%20with%20libFM.pdf

$y_i = w_0 + \sum_{j=1:P} w_{ij} x_{ij}  +  \sum_{j=1:P} \sum_{k > j} x_{ij} x_{i_k} < v_j v_k>$

статьи:

https://arxiv.org/pdf/1505.00641.pdf

https://www.csie.ntu.edu.tw/~b97053/paper/Factorization%20Machines%20with%20libFM.pdf

# рассмотрим библиотеку fastFM

http://ibayer.github.io/fastFM/index.html

In [2]:
from fastFM import als, sgd
import pandas as pd
import numpy as np

In [24]:
from sklearn.preprocessing import CategoricalEncoder
# pip install git+git://github.com/scikit-learn/scikit-learn.git
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [12]:
# информация о фильмах
items_info = pd.read_csv('ml-100k/u.item', header=None, sep='|', encoding="ISO-8859-1")

genres = [
    "Action", "Adventure", "Animation", "Children's", "Comedy", "Crime", "Documentary",
    "Drama", "Fantasy", "Film-Noir", "Horror", "Musical", "Mystery", "Romance", "Sci-Fi",
    "Thriller", "War", "Western"
]

items_info.columns = [
    "item_id", "movie title", "release date", "video release date", "IMDb URL", "unknown"
] + genres
items_info.head()

,item_id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [13]:
# информация о пользователях

def age_group(x):
    if x < 30:
        return 20
    elif x < 40:
        return 30
    else:
        return 40


users_info = pd.read_csv('ml-100k/u.user', header=None, sep='|')
users_info.columns = ['user_id', 'age', 'gender', 'occupation', 'zip_code']
users_info['age_gr'] = users_info.age.apply(age_group)
users_info.head()

,user_id,age,gender,occupation,zip_code,age_gr
0,1,24,M,technician,85711,20
1,2,53,F,other,94043,40
2,3,23,M,writer,32067,20
3,4,24,M,technician,43537,20
4,5,33,F,other,15213,30


In [14]:
def read_data(fname):
    data = pd.read_csv(fname, sep='\t', header=None)
    data.columns = ["user_id", "item_id", "rating", "timestamp"]
    return data

def merge_info(df):
    df = df.merge(users_info, on='user_id')
    df = df.merge(items_info, on='item_id')
    return df

In [19]:
df_test.timestamp.median()

883390350.0

In [20]:
(df_train.timestamp - df_test.timestamp.median()) / 60 / 60 / 24 / 12

0       -8.125571
1       -6.266569
2       -4.675338
3       -6.266619
4        6.135573
5        3.898170
6       -8.023523
7       -8.022633
8       -4.674777
9       -7.424028
10      -8.022847
11      -4.675338
12      -8.023288
13      -8.125621
14      -8.023478
15      -4.674777
16      -8.021944
17       3.898216
18      -8.023568
19      -4.675519
20      -8.022710
21      -8.022237
22      -8.023377
23      -8.023288
24      -8.022674
25      -6.266786
26      -8.022933
27      -4.675425
28      -4.675767
29      -8.022961
           ...   
90540   -7.608322
90541   -7.608250
90542    5.063366
90543    5.062992
90544    5.063520
90545    5.062816
90546    5.063116
90547    5.063251
90548   -7.608221
90549   -7.608543
90550    5.063539
90551    5.063405
90552    5.063458
90553    5.063620
90554    5.063499
90555   -7.607896
90556   -7.608089
90557   -7.608089
90558    5.114539
90559   -7.608291
90560    5.063055
90561    5.062948
90562   -7.607822
90563   -7.608270
90564    5

In [21]:
df_train = read_data('ml-100k/ua.base')
df_test = read_data('ml-100k/ua.test')

df_train['time'] = (df_train.timestamp - df_train.timestamp.median()) / 60 / 60 / 24 / 12
df_test['time']  = (df_test.timestamp - df_train.timestamp.median()) / 60 / 60 / 24 / 12
df_train.head()

,user_id,item_id,rating,timestamp,time
0,1,1,5,874965758,-7.569965
1,1,2,3,876893171,-5.710964
2,1,3,4,878542960,-4.119732
3,1,4,3,876893119,-5.711014
4,1,5,3,889751712,6.691179


In [22]:
df_train = merge_info(df_train)
df_test = merge_info(df_test)
df_train.head()

,user_id,item_id,rating,timestamp,time,age,gender,occupation,zip_code,age_gr,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,1,5,874965758,-7.569965,24,M,technician,85711,20,...,0,0,0,0,0,0,0,0,0,0
1,2,1,4,888550871,5.532960,53,F,other,94043,40,...,0,0,0,0,0,0,0,0,0,0
2,6,1,4,883599478,0.757311,42,M,executive,98101,40,...,0,0,0,0,0,0,0,0,0,0
3,10,1,4,877888877,-4.750599,53,M,lawyer,90703,40,...,0,0,0,0,0,0,0,0,0,0
4,13,1,3,882140487,-0.649895,47,M,educator,29206,40,...,0,0,0,0,0,0,0,0,0,0


In [26]:
enc = CategoricalEncoder(handle_unknown='ignore')

In [ ]:
# fit without users and items info
x_train = enc.fit_transform(df_train[['user_id', 'item_id']]).tocsc()
x_test = enc.transform(df_test[['user_id', 'item_id']]).tocsc()

y_train = df_train['rating']
y_test = df_test['rating']

In [ ]:
fm_simple = als.FMRegression(
    rank=5,
    n_iter=100,
    l2_reg=10,
    init_stdev=0.01
)
fm_simple.fit(x_train, y_train)

In [ ]:
pred_simple = fm_simple.predict(x_test)
print('sklearn MAE: {:.4f}'.format(mean_absolute_error(y_test, pred_simple)))
print('sklearn RMSE: {:.4f}'.format(np.sqrt(mean_squared_error(y_test, pred_simple))))

In [ ]:
fm_simple2 = sgd.FMRegression(
    rank=10,
    step_size=0.005,
    n_iter=5000000,
    l2_reg=0.02,
    init_stdev=0.01)
fm_simple2.fit(x_train, y_train)

In [ ]:
pred_simple2 = fm_simple2.predict(x_test)
print('sklearn MAE: {:.4f}'.format(mean_absolute_error(y_test, pred_simple2)))
print('sklearn RMSE: {:.4f}'.format(np.sqrt(mean_squared_error(y_test, pred_simple2))))

In [ ]:
print('global mean: {:.2f}'.format(fm_simple.w0_))

In [ ]:
fm_simple.w_

In [ ]:
fm_simple.V_[0]

# учтем фичи пользователей и фильмов

In [27]:
from scipy import sparse

In [28]:
# fit with users and items info
x_train_attr = enc.fit_transform(df_train[['user_id', 'item_id', 'age_gr', 'gender']]).tocsc()
x_test_attr = enc.transform(df_test[['user_id', 'item_id', 'age_gr', 'gender']]).tocsc()

# items information
items_train = sparse.csc_matrix(df_train[genres].values)
items_test = sparse.csc_matrix(df_test[genres].values)

# time information
time_train = sparse.csc_matrix(df_train['time'].values.reshape(-1, 1))
time_test = sparse.csc_matrix(df_test['time'].values.reshape(-1, 1))


x_train_attr = sparse.hstack([x_train_attr, items_train])
x_test_attr = sparse.hstack([x_test_attr, items_test])

x_train_attr_time = sparse.hstack([x_train_attr, items_train, time_train])
x_test_attr_time = sparse.hstack([x_test_attr, items_test, time_test])

print(x_train_attr.shape, x_test_attr.shape)

NameError: name 'enc' is not defined

In [ ]:
fm = als.FMRegression(
    rank=5,
    n_iter=100,
    l2_reg=10,
    init_stdev=0.01
)

fm.fit(x_train_attr, y_train)

In [ ]:
pred_attr = fm.predict(x_test_attr)
print('sklearn MAE: {:.4f}'.format(mean_absolute_error(y_test, pred_attr)))
print('sklearn RMSE: {:.4f}'.format(np.sqrt(mean_squared_error(y_test, pred_attr))))

In [ ]:
fm2 = als.FMRegression(
    rank=5,
    n_iter=100,
    l2_reg=10,
    init_stdev=0.01
)

fm2.fit(x_train_attr_time, y_train)

In [ ]:
pred_attr = fm2.predict(x_test_attr_time)
print('sklearn MAE: {:.4f}'.format(mean_absolute_error(y_test, pred_attr)))
print('sklearn RMSE: {:.4f}'.format(np.sqrt(mean_squared_error(y_test, pred_attr))))

## Эффект различных слагаемых на качество предсказания рейтинга



<img src='effect_factorizations.png'>

источник:

https://datajobs.com/data-science-repo/Recommender-Systems-[Netflix].pdf

# Оценка качества рекомендательной системы

* Качество рейтингов
    * MAE, MSE
* Качество событий
    * F-score, ROC-AUC, PR-AUC
* Качество ранжирования
    * precision@k, recall@k
    * Mean average precision@k
    * $DCG@k(u) = \sum\limits_{p=1}^k \frac{val(i,p)}{\log{(p+1)}}$
    * $nDCG@k(u) = \frac{DCG@k(u)}{\max{(DCG@k(u))}}$


ссылки:

https://en.wikipedia.org/wiki/Evaluation_measures_(information_retrieval)#Offline_metrics

https://habr.com/company/econtenta/blog/303458/

# Ассоциативные правила

Рекомендации вида: «Кто купил x, также купил y».

Дано: база транзакций / покупок

Хотим посчитать: 
- P(X ...) - support(X ...)
- P(Y|X) - сonfidence(X->Y)
- Lift(X->Y) = support(X and Y) / (support(X) * support(Y))
- Conviction(X->Y) = (1 - support(Y)) / (1 - confidence(X->Y))

где X, Y - товары


Эффективный рассчет величин выше: Apriori Algorithm


Статья:

https://habr.com/company/ods/blog/353502/

Полезные ссылки:
1. https://habrahabr.ru/company/surfingbird/blog/139518/
2. https://d4datascience.wordpress.com/category/predictive-analytics/
3. Data Science from scratch
4. https://habrahabr.ru/company/yandex/blog/241455/
5. https://www.kaggle.com/rounakbanik/movie-recommender-systems/notebook
6. http://www.cs.ubbcluj.ro/~gabis/DocDiplome/SistemeDeRecomandare/Recommender_systems_handbook.pdf
7. https://www.coursera.org/specializations/recommender-systems
8. https://www.cs.umd.edu/~samir/498/Amazon-Recommendations.pdf
9. https://datajobs.com/data-science-repo/Recommender-Systems-[Netflix].pdf
10. http://surprise.readthedocs.io/en/stable/matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.SVD
11. https://arxiv.org/pdf/1505.00641.pdf
12. https://www.csie.ntu.edu.tw/~b97053/paper/Factorization%20Machines%20with%20libFM.pdf
13. https://habr.com/company/ods/blog/353502/